In [172]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as pltcolors
import numpy as np
import plotly.express as px
import plotly
import activitiesdf
from datetime import timedelta
import importlib
importlib.reload(activitiesdf)

<module 'activitiesdf' from '/Users/kristofferek/Documents/git/mscthesis/src/py/activitiesdf.py'>

In [183]:
activities = activitiesdf.read_csv("sweden")
activities

,userid,region,tweetid,createdat,latitude,longitude,month,weekday,hourofday,timezone,geometry,tweet_count,quantile,label
0,5616,0,306814580301119488,2013-02-27 18:14:30+01:00,57.599221,18.436371,2.0,3.0,18.0,Europe/Stockholm,POINT (18.43637 57.59922),2171,"(2076.0, 4778.0]",other
1,5616,0,596331871241379840,2015-05-07 17:12:52+02:00,57.599221,18.436371,5.0,4.0,17.0,Europe/Stockholm,POINT (18.43637 57.59922),2171,"(2076.0, 4778.0]",other
2,5616,0,623057810864111616,2015-07-20 11:12:12+02:00,57.599221,18.436371,7.0,1.0,11.0,Europe/Stockholm,POINT (18.43637 57.59922),2171,"(2076.0, 4778.0]",other
3,5616,0,679673567416565760,2015-12-23 15:43:00+01:00,57.599221,18.436371,12.0,3.0,15.0,Europe/Stockholm,POINT (18.43637 57.59922),2171,"(2076.0, 4778.0]",other
4,5616,0,757647103011262465,2016-07-25 20:42:00+02:00,57.599221,18.436371,7.0,1.0,20.0,Europe/Stockholm,POINT (18.43637 57.59922),2171,"(2076.0, 4778.0]",other
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1276010,806822309478727680,7,815619064613609472,2017-01-01 19:01:53+01:00,57.716700,12.916700,1.0,0.0,19.0,Europe/Stockholm,POINT (12.91670 57.71670),44,"(4.999, 179.0]",other
1276011,806822309478727680,7,815835652486938624,2017-01-02 09:22:31+01:00,57.716700,12.916700,1.0,1.0,9.0,Europe/Stockholm,POINT (12.91670 57.71670),44,"(4.999, 179.0]",other
1276012,806822309478727680,7,815937364736380928,2017-01-02 16:06:41+01:00,57.716700,12.916700,1.0,1.0,16.0,Europe/Stockholm,POINT (12.91670 57.71670),44,"(4.999, 179.0]",other
1276013,806822309478727680,8,816003760006668288,2017-01-02 20:30:31+01:00,57.730060,12.942200,1.0,1.0,20.0,Europe/Stockholm,POINT (12.94220 57.73006),44,"(4.999, 179.0]",other


In [184]:
activities['createdat'] = pd.to_datetime(activities.createdat, utc=True)

In [185]:
acts = activities.set_index(['userid', 'region'])

### Add yearmonth column

In [186]:
acts['yearmonth'] = acts['createdat'].dt.to_period('M')


/Users/kristofferek/miniconda3/envs/mscthesis/lib/python3.8/site-packages/pandas/core/arrays/datetimes.py:1099: UserWarning:

Converting to PeriodArray/Index representation will drop timezone information.



## Label multiple home locations

In [ ]:
acts_2 = activitiesdf.label_multiple_homes(acts)
acts_2

copying
reindex
largest cluster
region
134    121
252    115
0      114
dtype: int64


## Label work location

In [ ]:
acts_3 = activitiesdf.label_work(acts_2)

## Visualise a user's labels and clusters

In [ ]:
#18192787
#14079762
#85666280
user = acts_3.loc[85666280].reset_index().set_index('region')
homestr = 'Home: '+ str(np.unique(user[user['label'] == 'home'].index.values))
workstr = 'Work: '+ str(np.unique(user[user['label'] == 'work'].index.values))


### Extract the user's 10 largest clusters

In [ ]:
topregions = user.groupby(['region']).size().sort_values(ascending=False).head(10)
topten = user.loc[topregions.index.values]

## Plot cluster sizes over time (monthly)

In [ ]:
bars = topten.groupby(['yearmonth', 'region']).size().unstack().fillna(0).plot.bar(stacked=True, figsize=(20, 10), width=0.95)
bars.set_title("Top 10 clusters for a user over time")
print(homestr)
print(workstr)

## Scatter plot clusters on a map

In [ ]:
plotly.offline.init_notebook_mode(connected=True)

In [ ]:
fig = px.scatter_mapbox(topten.reset_index(), lat='latitude', lon='longitude', hover_data=['hourofday', 'weekday', 'region'], color='label')
fig.update_layout(mapbox_style="open-street-map")
fig.show()